## 3 Datasets:
- MNIST
- Spambase
- 20NG

For each, perform 2 classification algorithms:
- L2-reg Logistic Regression
- Decision Trees


allowed to use library for classification. 

allowed to use library to process data in appropriate formats.

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch as torch
from sklearn.datasets import _twenty_newsgroups
from torchvision import datasets, transforms

# MNIST

In [8]:
# Load the MNIST dataset
mnist_dataset = datasets.MNIST(root='data', download=True, transform=transforms.ToTensor())
features = mnist_dataset.data
labels = mnist_dataset.targets

# Normalize the features
features = features / 255.0



In [9]:
# using pytorch to perform logistic regression:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))
    
# Initialize the model, loss function, and optimizer
input_dim = 28 * 28  # MNIST images are 28x28 pixels
output_dim = 10  # 10 classes for digits 0-9
model = LogisticRegression(input_dim, output_dim)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
# Prepare the data for training
features = features.view(-1, input_dim).float()
labels = labels.long()
# Training loop
num_epochs = 5
batch_size = 64
num_batches = len(features) // batch_size
for epoch in range(num_epochs):
    for i in range(num_batches):
        # Get the batch data
        batch_features = features[i * batch_size:(i + 1) * batch_size]
        batch_labels = labels[i * batch_size:(i + 1) * batch_size]
        
        # Forward pass
        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
with torch.no_grad():
    correct = 0
    total = 0
    for i in range(num_batches):
        batch_features = features[i * batch_size:(i + 1) * batch_size]
        batch_labels = labels[i * batch_size:(i + 1) * batch_size]
        
        outputs = model(batch_features)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()
    accuracy = correct / total
print(f'Accuracy of the model on the MNIST test images: {accuracy * 100:.2f}%')


Epoch [1/5], Loss: 2.0230
Epoch [2/5], Loss: 1.9189
Epoch [3/5], Loss: 1.8643
Epoch [4/5], Loss: 1.8293
Epoch [5/5], Loss: 1.8042
Accuracy of the model on the MNIST test images: 84.18%


In [ ]:
# Using decision tree classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Load the MNIST dataset
mnist_dataset_train = datasets.MNIST(root='data', train = True, download=True)

mnist_dataset = datasets.MNIST(root='data', train = False, download=True)
print(mnist_dataset_train.data.shape)
print(mnist_dataset.data.shape)
features = mnist_dataset.data
labels = mnist_dataset.targets

# convert to numpy arrays:
features = features.numpy()
labels = labels.numpy()

# Normalize the features
features = features / 255.0

# Reshape the features to 2D array
features = features.reshape(features.shape[0], -1)

# Initialize the Decision Tree Classifier
dt_classifier = DecisionTreeClassifier()

# Train the Decision Tree Classifier
dt_classifier.fit(features, labels)

# Predict on the training set
dt_predictions = dt_classifier.predict(features)
# Calculate accuracy
dt_accuracy = accuracy_score(labels, dt_predictions)
print(f'Accuracy of the Decision Tree Classifier on the MNIST dataset: {dt_accuracy * 100:.2f}%')



torch.Size([60000, 28, 28])
torch.Size([60000, 28, 28])
Accuracy of the Decision Tree Classifier on the MNIST dataset: 100.00%


# Spambase


In [10]:
spambase = pd.read_csv('spambase.data', header=None, sep=',')
spambase

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


In [11]:
spambase = spambase.sample(frac=1).reset_index(drop=True)
spambase = spambase.values
spambase

array([[1.80e-01, 6.00e-02, 2.40e-01, ..., 2.90e+01, 8.43e+02, 0.00e+00],
       [0.00e+00, 0.00e+00, 0.00e+00, ..., 1.70e+01, 5.30e+01, 0.00e+00],
       [0.00e+00, 0.00e+00, 0.00e+00, ..., 3.00e+00, 1.30e+01, 0.00e+00],
       ...,
       [0.00e+00, 0.00e+00, 2.60e-01, ..., 1.50e+01, 1.88e+02, 1.00e+00],
       [1.24e+00, 0.00e+00, 0.00e+00, ..., 1.55e+02, 2.71e+02, 1.00e+00],
       [0.00e+00, 0.00e+00, 1.70e-01, ..., 2.40e+01, 5.40e+02, 0.00e+00]])

# 20NG

In [12]:
newsgroups = _twenty_newsgroups.fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
features = newsgroups.data
labels = newsgroups.target